데이터를 한국 단어로 바꾸면 어떻게 될까? - 아이유 중심으로

네이버 카페 대본 모아놓은 사이트 있나?




In [2]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!
import os
import glob
from sklearn.model_selection import train_test_split

txt_file_path = os.getenv('HOME')+'/aiffel/exp/exp6lyrics/lyriclist/data/lyrics/song_lyrics/*'
txt_list = glob.glob(txt_file_path)

In [3]:
raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['When somebody loved me', 'Everything was beautiful', 'Every hour we spent together']


In [4]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

When somebody loved me
Everything was beautiful
Every hour we spent together
Lives within my heart And when she was sad
I was there to dry her tears
And when she was happy so was I
When she loved me Through the summer and the fall
We had each other that was all
Just she and I together
Like it was meant to be And when she was lonely


In [5]:
def preprocess_sentence(sentence):
    
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.


<start> this is sample sentence . <end>


In [6]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    #if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> when somebody loved me <end>',
 '<start> everything was beautiful <end>',
 '<start> every hour we spent together <end>',
 '<start> lives within my heart and when she was sad <end>',
 '<start> i was there to dry her tears <end>',
 '<start> and when she was happy so was i <end>',
 '<start> when she loved me through the summer and the fall <end>',
 '<start> we had each other that was all <end>',
 '<start> just she and i together <end>',
 '<start> like it was meant to be and when she was lonely <end>']

In [7]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2  46 265 ...   0   0   0]
 [  2 182  57 ...   0   0   0]
 [  2 133 925 ...   0   0   0]
 ...
 [  2   1   5 ...   0   0   0]
 [  2 631   4 ...   0   0   0]
 [  2   1  71 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f50d35aefd0>


In [8]:
# 생성된 텐서 데이터를 3번째 행, 10번째 열까지만 출력해 봅시다.

print(tensor[:3, :10])

[[   2   46  265  589   12    3    0    0    0    0]
 [   2  182   57  424    3    0    0    0    0    0]
 [   2  133  925   23 1133  282    3    0    0    0]]


In [9]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [10]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

#print(src_input[0])
#print(tgt_input[0])
print(src_input.shape)
print(tgt_input.shape)

(175986, 346)
(175986, 346)


In [14]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

NameError: name 'dataset' is not defined

In [13]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140788, 346)
Target Train: (140788, 346)


In [ ]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset



In [11]:
print(len(raw_corpus_15token))
print(len(raw_corpus_15token[0]), raw_corpus_15token[0])
print(len(raw_corpus_15token[1]), raw_corpus_15token[1])
print(len(raw_corpus_15token[2]), raw_corpus_15token[2])
print(len(raw_corpus_15token[3]), raw_corpus_15token[3])
print(len(raw_corpus_15token[4]), raw_corpus_15token[4])

NameError: name 'raw_corpus_15token' is not defined

In [ ]:
enc_train, enc_val, dec_train, dec_val = train_test_split(raw_corpus, test_size=0.2, random_state=23)

In [ ]:
X_train_dig, X_test_dig, y_train_dig, y_test_dig = train_test_split(hand_digits_data, 
                                                    hand_digits_label, 
                                                    test_size=0.2, 
                                                    random_state=7)

print('X_train 개수: ', len(X_train_dig), ', X_test 개수: ', len(X_test_dig))

In [ ]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

In [ ]:
#Loss

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [ ]:
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)